In [35]:
from zenq.clvmodels import modeling

In [36]:
from zenq.api import endpoints

In [37]:
from zenq.clvmodels import gamma

In [38]:
from zenq.api import prepare_db

In [39]:
from zenq.visualizations import plot

In [40]:
from zenq import utils

In [41]:
plot.PlotUtils.pu()

ok
None


In [52]:

prepare_db.db.main()

Initializing the database.. 

ProgrammingError: (psycopg2.errors.InvalidSchemaName) schema "initial" does not exist
LINE 2: CREATE TABLE initial."Location" (
                     ^

[SQL: 
CREATE TABLE initial."Location" (
	id SERIAL NOT NULL, 
	location_id VARCHAR(50) NOT NULL, 
	location_name VARCHAR(50) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (location_id), 
	UNIQUE (location_name)
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [32]:
#from zenq.api import openapi_main